In [3]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolExecutor, ToolNode
from langchain_core.tools import tool

# -----------------------------
# Define Tools
# -----------------------------
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


# -----------------------------
# Define the ReAct Agent State
# -----------------------------
class AgentState(dict):
    """State passed between nodes."""
    input: str
    scratchpad: str
    output: str


# -----------------------------
# Setup Model & Tools
# -----------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [multiply, add]
tool_executor = ToolExecutor(tools)


# -----------------------------
# Define Agent Node (ReAct loop)
# -----------------------------
def agent_node(state: AgentState):
    """Decide next action: Reasoning or Acting."""
    # Format input for LLM with reasoning + history
    prompt = f"""
You are an agent that follows the ReAct pattern:
- First, reason step by step
- Then decide an action, or return final answer.

Question: {state["input"]}
Scratchpad so far: {state.get("scratchpad", "")}
"""
    response = llm.invoke(prompt)

    text = response.content

    if "Action:" in text:
        # Extract tool + input
        # Example format: Action: multiply(3, 5)
        action_line = text.split("Action:")[1].strip()
        tool_name, args_str = action_line.split("(", 1)
        tool_name = tool_name.strip()
        args = args_str.strip(") ")

        # For simplicity, assume args are comma-separated ints
        args = [int(x.strip()) for x in args.split(",")]

        result = tool_executor.invoke({"tool": tool_name, "tool_input": args})
        new_scratchpad = state.get("scratchpad", "") + f"\nUsed {tool_name} → {result}"
        return {**state, "scratchpad": new_scratchpad}

    elif "Final Answer:" in text:
        final_answer = text.split("Final Answer:")[1].strip()
        return {**state, "output": final_answer}

    return state


# -----------------------------
# Build Graph
# -----------------------------
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_node)

workflow.add_edge(START, "agent")
workflow.add_edge("agent", "agent")  # self-loop until "Final Answer"
workflow.add_edge("agent", END)

app = workflow.compile()


# -----------------------------
# Run the Agent
# -----------------------------
if __name__ == "__main__":
    user_question = "What is (3 + 5) * 2 ?"
    final_state = app.invoke({"input": user_question})
    print("Final Output:", final_state["output"])
